In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', None)
SEED=27

## Загрузка сырых данных

In [2]:
def get_3d_quartile_words_count(sentences_array):
    """
    Получает массив строк/предложений.
    Отдает число N.
    75% всех предложений данного массива не превышают N слов в длину.
    """
    def count_words(sentence):
        return len(sentence.split(' '))
        
    
    return round(
        np.percentile(sentences_array.apply(count_words), 75)
    )

orig_toxic_comments_df = pd.read_csv('data/toxicCommentsOriginalDF.csv')
print('Количество комментариев:', len(orig_toxic_comments_df))
print(f"75% всех комментариев не больше {get_3d_quartile_words_count(orig_toxic_comments_df['comment'])} слов в длину")
orig_toxic_comments_df.head(3)

Количество комментариев: 14412
75% всех комментариев не больше 32 слов в длину


,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.\n",1.0
2,Собаке - собачья смерть\n,1.0


## Split train-validation-test DF

In [3]:
df = orig_toxic_comments_df.copy()

y = df.pop('toxic')
X = np.array(df)

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

## Train model

In [5]:
import tensorflow as tf

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers, losses

print(tf.__version__)

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
2.5.0-rc0


In [6]:
from textPreprocessing import preprocess_text

In [7]:
MAX_FEATURES = 10000
MAX_SEQUENCE_LENGTH = get_3d_quartile_words_count(df['comment'])


def decode_byte_string_to_russian_text(string_tensor):
    try:
        return string_tensor[0].numpy().decode('utf-8')
    except:
        return ''

def custom_standardization(texts_tensor):    
    result = tf.map_fn(fn=lambda x: preprocess_text(decode_byte_string_to_russian_text(x)), elems=texts_tensor)
    
    return result

vectorize_layer = TextVectorization(
#     standardize=custom_standardization,
    max_tokens=MAX_FEATURES,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH
)
vectorize_layer.adapt(X_train)

In [8]:
model = tf.keras.Sequential([
    vectorize_layer,
    layers.Embedding(
        input_dim = len(vectorize_layer.get_vocabulary()),
        output_dim = 16
    ),
    layers.Dropout(0.2),
    layers.GlobalAveragePooling1D(),
    layers.Dropout(0.2),
    layers.Dense(1),
    layers.Activation('sigmoid') # то есть в конце мы выдаем уже вероятности
])

model.compile(
    loss=losses.BinaryCrossentropy(from_logits=False),
    optimizer='adam',
#     metrics=[tf.keras.metrics.Accuracy]
    metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Recall(), tf.keras.metrics.Precision()]
)


In [9]:
epochs = 20
history = model.fit(
    x=X_train,
    y=y_train,
    epochs=epochs,
    validation_split=0.2, # сколько от тестовой выборки отрезать под валидационную
)

Epoch 1/20
289/289 [==============================] - 2s 3ms/step - loss: 0.6691 - binary_accuracy: 0.6562 - recall: 0.0252 - precision: 0.3903 - val_loss: 0.6173 - val_binary_accuracy: 0.6670 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 2/20
289/289 [==============================] - 1s 2ms/step - loss: 0.6103 - binary_accuracy: 0.6611 - recall: 0.0020 - precision: 0.3486 - val_loss: 0.5768 - val_binary_accuracy: 0.6722 - val_recall: 0.0195 - val_precision: 0.8333
Epoch 3/20
289/289 [==============================] - 1s 2ms/step - loss: 0.5657 - binary_accuracy: 0.6868 - recall: 0.0804 - precision: 0.9275 - val_loss: 0.5372 - val_binary_accuracy: 0.7559 - val_recall: 0.3346 - val_precision: 0.8317
Epoch 4/20
289/289 [==============================] - 1s 2ms/step - loss: 0.5144 - binary_accuracy: 0.7635 - recall: 0.3614 - precision: 0.8655 - val_loss: 0.4986 - val_binary_accuracy: 0.7827 - val_recall: 0.4193 - val_precision: 0.8541
Epoch 5/20
289/289 [====================

In [10]:
loss, accuracy, recall, precision = model.evaluate(x=X_test, y=y_test)

print("Loss: ", loss)
print("Accuracy: ", accuracy)
print("Recall: ", recall)
print("Precision: ", precision)

91/91 [==============================] - 0s 903us/step - loss: 0.3499 - binary_accuracy: 0.8661 - recall: 0.7746 - precision: 0.8121
Loss:  0.3498672842979431
Accuracy:  0.866111695766449
Recall:  0.7746331095695496
Precision:  0.812087893486023


In [11]:
tf.math.confusion_matrix(labels=y_test, predictions=(model.predict(X_test) > 0.5) * 1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1758,  171],
       [ 215,  739]], dtype=int32)>